# Eindhoven University of Technology, Netherlands.
## Mathematics & Computer Science
## Data Science Master track
## Recommender Systems
Lecturer: Dr. Vlado Menkovski
- Franziska Boenisch
- Adriano Cardace 
- Camilo Montenegro Hernandez
## Image Caption Generation

## Task 2.3: Discussion

1. Briefly describe one of the preceding works on modeling Image-Caption according to the paper and its limitation. Name the advantage(s) of the current Image-Caption generator as compared to the previous work?


2. Given an image with multiple text descriptions, how to represent this pair of data as model input-output?

- Explain the answer in a notation (x, y) or clear description that guarantees you can run the same model with your input-output representation. Name all the input(s) and output(s) of the model.
- How does the model extract image features from raw images? Name and briefly explain how to employ the feature extractor. Include the dimension (array shape) of the extracted features.
- How are the train descriptions represented into the model? Why do we need to add “starting” and “ending” token of every caption in a preprocessing stage?
- How does the model initialize RNN states for caption generator? What is being fed into the decoder states in the first time step (t = 0)?

3. What is the motivation of incorporating Beam Search in Sequence-to-Sequence learning? Briefly explain how the method works in an inference stage.

### YOUR ANSWER

1. We chose to present the work by Mao et al. [https://arxiv.org/pdf/1410.1090.pdf] because according to the paper, it is the most similar work. 

Brief description of Mao's approach: They use a multimodal Recurrent Neural Network (m-RNN). The model consists of two sub-networks:  a deep recurrent neural network for sentences and a deep convolutional network for images. These two sub-networks interact with each other in a multimodal layer to form the whole m-RNN model. This is the same encoder-decoder idea also used in the paper we read in class.

The differences of the current Image-Caption generator compared to this previous work are, according to the paper, that here, they use a much more powerful RNN (compared to Mao). According to  the paper we read in class , NIC provides the visual input to the RNN directly. Mao does not do it. This leads to the advantage that  for the RNN it becomes possible to keep track of objects that have been explained by the text. As a result, the model achives substantially better results as the one of Mao.

2. Given an image with multiple descriptions, 

a) During training, we provide the captions for the 5 sentences that we have per image in the following way: 
Example 1: "the cat is sitting."
x1 = startseq the cat is sitting
y1 = the cat is sitting endseq


Example 2: "the dog is eating."
x1 = startseq the dog is eating
y1 = the dog is eating endseq

And we also include the image as the input (belonging to x_i). Thereby the model learns to predict the following word given the previous words, but also taking into account the image.

b) To extract features from the images, we use a CNN that is originally used for classification. In this assignment, we are not doing that preprcessing ourselves, but we are directly loading the data. Looking at the preprocessing file 0.1, we see that the nework used is InceptionV3. The last two layers are deleted. These layers are used for the classification, but we are only interested in the layers before because they contain the features.
We are appending one Dense layer to the remaining network to get a smaller representation of the features. The features we load have dimension 2048, and we tried first scaling them to 300. 

c) To represent the captions to the model, we first have to tokenize them. This is done in the preprocessing. Then, we build a vocabulary, so that every word is an index. Those indices need to be passed through an embedding layer to get their representations which can be fed to the model.

We need to add start tokens, because the model needs at each timestep a image feature input and a previous word. At the beginning, there is no previous word, hence, the startseq is a dummy word we can feed at the first step.
We need to add end tokens, because the model needs to learn when to stop a sentence. If it kept on predicting without learning a stop, all sentences would have the same (maximum length) and this is not what we want.

d) The model initializes RNN states as the feature representation of the image. In the training phase the image representation and a complete representation of the input sequence (as output of the encoder RNN) is fed into the decoder states in the first time step (t = 0). During inference phase, we provide the representation of the image and the previous word (startseq as a t=0 first timestep).

3. Beam search is an optimization of the best-first search (like e.g. the A* algorithm that explores a graph by expanding the most promising node chosen according to a specified rule) that reduces the memory requirements. Hence, Beam search is always expanding the most promising node. In contrast to best-first search, only a predefined number of best partial solutions are kept as candidates.
 
 When using Beam search, we define a parameter B (beam width), which represents the number of words we want the algorithm to consider as the first word. Then, having picked the B number of choices for the first word, for each of these choices the algorithm considers what could be the second word. The goal is to find the most likely pair of the first and second word. Then we go on over all following words the same way. The goal is trying to maximize the product of the probabilities over the whole sentence. 
 
 This leads to explaining the motivation behind using Beam search, which is, that we do not want to pick word-by-word, but by probability over the entire sentence. If we were to choose the first word greedily without considerin the rest of the sentence, we might find the most probable word, however, the rest of the sentence might not be ideal. This can be overcome if we look at the probabilities over the entire sentence.
 
